<a href="https://colab.research.google.com/github/SnehaVenkatesh19/Fraud-Detection/blob/main/fraud_detection_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

file_path = '/content/drive/MyDrive/Datasets/creditcard.csv'
df = pd.read_csv(file_path)

df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
print("Shape:", df.shape)

# Check column info
print(df.info())

# Check class distribution
print(df['Class'].value_counts(normalize=True) * 100)

Shape: (284807, 31)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  flo

Scaling time and amount and splitting data

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    confusion_matrix, classification_report, roc_auc_score,
    precision_recall_curve, average_precision_score
)

# Features (drop target)
X = df.drop(columns=["Class"])
y = df["Class"].astype(int)

# Scale Time & Amount (rest are PCA components already)
scaler = StandardScaler()
X[["Time", "Amount"]] = scaler.fit_transform(X[["Time", "Amount"]])

# Stratified split to preserve fraud ratio
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

print("Train shape:", X_train.shape, " Test shape:", X_test.shape)
print("Fraud % in train:", y_train.mean()*100, " Fraud % in test:", y_test.mean()*100)


Train shape: (227845, 30)  Test shape: (56962, 30)
Fraud % in train: 0.17292457591783889  Fraud % in test: 0.17204452090867595


Applying SMOTE resampling for logistic and random forest

In [6]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE only on training data
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

print("Before SMOTE:", y_train.value_counts())
print("After SMOTE:", y_train_res.value_counts())

# ---------------------------
# Logistic Regression + SMOTE
# ---------------------------
logit_smote = LogisticRegression(max_iter=500, class_weight=None, n_jobs=None)
logit_smote.fit(X_train_res, y_train_res)

proba_lr_smote = logit_smote.predict_proba(X_test)[:, 1]
pred_lr_smote = (proba_lr_smote >= 0.5).astype(int)

print("\n=== Logistic Regression with SMOTE @ 0.5 threshold ===")
print("ROC AUC:", roc_auc_score(y_test, proba_lr_smote))
print("PR AUC:", average_precision_score(y_test, proba_lr_smote))
print(confusion_matrix(y_test, pred_lr_smote))
print(classification_report(y_test, pred_lr_smote, digits=4))

# ---------------------------
# Random Forest + SMOTE
# ---------------------------
rf_smote = RandomForestClassifier(
    n_estimators=400,
    class_weight=None,   # don’t use class weights with SMOTE
    n_jobs=-1,
    random_state=42
)
rf_smote.fit(X_train_res, y_train_res)

proba_rf_smote = rf_smote.predict_proba(X_test)[:, 1]
pred_rf_smote = (proba_rf_smote >= 0.5).astype(int)

print("\n=== Random Forest with SMOTE @ 0.5 threshold ===")
print("ROC AUC:", roc_auc_score(y_test, proba_rf_smote))
print("PR AUC:", average_precision_score(y_test, proba_rf_smote))
print(confusion_matrix(y_test, pred_rf_smote))
print(classification_report(y_test, pred_rf_smote, digits=4))


Before SMOTE: Class
0    227451
1       394
Name: count, dtype: int64
After SMOTE: Class
0    227451
1    227451
Name: count, dtype: int64

=== Logistic Regression with SMOTE @ 0.5 threshold ===
ROC AUC: 0.9698482164390798
PR AUC: 0.7248611768135257
[[55406  1458]
 [    8    90]]
              precision    recall  f1-score   support

           0     0.9999    0.9744    0.9869     56864
           1     0.0581    0.9184    0.1094        98

    accuracy                         0.9743     56962
   macro avg     0.5290    0.9464    0.5481     56962
weighted avg     0.9982    0.9743    0.9854     56962


=== Random Forest with SMOTE @ 0.5 threshold ===
ROC AUC: 0.9738480032558887
PR AUC: 0.8765519855576973
[[56850    14]
 [   16    82]]
              precision    recall  f1-score   support

           0     0.9997    0.9998    0.9997     56864
           1     0.8542    0.8367    0.8454        98

    accuracy                         0.9995     56962
   macro avg     0.9269    0.9182    0

In [8]:
!pip -q install xgboost shap imbalanced-learn
import xgboost as xgb
import shap


XGBoost with threshold tuning

In [11]:
# Compute class weight from training data
pos = int(y_train.sum())
neg = int(len(y_train) - pos)
scale_pos_weight = neg / pos
print("scale_pos_weight:", round(scale_pos_weight, 2))

# XGBoost model (no SMOTE)
xgb_clf = xgb.XGBClassifier(
    n_estimators=600,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    min_child_weight=1,
    eval_metric="aucpr",
    scale_pos_weight=scale_pos_weight,
    tree_method="hist",
    random_state=42,
    n_jobs=-1
)

xgb_clf.fit(X_train, y_train)
proba_xgb = xgb_clf.predict_proba(X_test)[:, 1]
pred_xgb_05 = (proba_xgb >= 0.5).astype(int)

print("=== XGBoost @ 0.5 threshold ===")
print("ROC AUC:", roc_auc_score(y_test, proba_xgb))
print("PR AUC:", average_precision_score(y_test, proba_xgb))
print(confusion_matrix(y_test, pred_xgb_05))
print(classification_report(y_test, pred_xgb_05, digits=4))


scale_pos_weight: 577.29
=== XGBoost @ 0.5 threshold ===
ROC AUC: 0.9818813129500534
PR AUC: 0.8768947972755641
[[56849    15]
 [   17    81]]
              precision    recall  f1-score   support

           0     0.9997    0.9997    0.9997     56864
           1     0.8438    0.8265    0.8351        98

    accuracy                         0.9994     56962
   macro avg     0.9217    0.9131    0.9174     56962
weighted avg     0.9994    0.9994    0.9994     56962



In [10]:
prec_xgb, rec_xgb, thr_xgb = precision_recall_curve(y_test, proba_xgb)
f1_xgb = 2 * (prec_xgb * rec_xgb) / (prec_xgb + rec_xgb + 1e-9)
best_idx_xgb = np.argmax(f1_xgb)
best_thr_xgb = thr_xgb[best_idx_xgb] if best_idx_xgb < len(thr_xgb) else 0.5

print("Best XGB threshold by F1:", best_thr_xgb)

pred_xgb_tuned = (proba_xgb >= best_thr_xgb).astype(int)
print(confusion_matrix(y_test, pred_xgb_tuned))
print(classification_report(y_test, pred_xgb_tuned, digits=4))


Best XGB threshold by F1: 0.9277023
[[56856     8]
 [   18    80]]
              precision    recall  f1-score   support

           0     0.9997    0.9999    0.9998     56864
           1     0.9091    0.8163    0.8602        98

    accuracy                         0.9995     56962
   macro avg     0.9544    0.9081    0.9300     56962
weighted avg     0.9995    0.9995    0.9995     56962

